In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import  TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
path='/content/gdrive/MyDrive/Datasets/'
movies_df =pd.read_csv(path+'movies.csv', dtype={'movieId': 'int32', 'title': 'str'})
ratings_df= pd.read_csv(path+'ratings.csv',usecols=['userId', 'movieId', 'rating'],
            dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [ ]:
movies_df.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
movies_df['title']= movies_df['title'].apply(lambda x: ' '.join(x.split()[:-1]))


In [ ]:
ratings_df.head()


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
df = pd.merge(ratings_df,movies_df,on='movieId')
df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story,Adventure|Animation|Children|Comedy|Fantasy


In [ ]:
df['rating'] = df['rating'].fillna(0)

In [ ]:
# Create a TF-IDF matrix for movie genres
tfidf= TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['genres'])

# Calculate cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Create an index mapping movie titles to movie indices
indices = pd.Series(movies_df.index, index=movies_df['title']).drop_duplicates()



In [ ]:
def get_content_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    sim_scores = enumerate(cosine_sim[idx])
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    sim_index = [i[0] for i in sim_scores]
    #recommended_movies = movies_df['title'].iloc[sim_index]

    recommended_movies = movies_df.iloc[sim_index]  # Use iloc to access both title and genre

    recommended_movie_titles = recommended_movies["title"].values.tolist()
    recommended_movie_genres = recommended_movies["genres"].values.tolist()


    return recommended_movie_titles, recommended_movie_genres




# Example usage
title='Tangled'
# Call the get_content_recommendations function to get the recommended movie information
recommended_titles, recommended_genres = get_content_recommendations(title)

print("Recommended Movies:")

for idx, (movie, genre) in enumerate(zip(recommended_titles, recommended_genres), start=1):
    print(f"{idx}.{movie},{genre}")



Recommended Movies:
1.Beauty and the Beast,Animation|Children|Fantasy|Musical|Romance|IMAX
2.Fantasia 2000,Animation|Children|Musical|IMAX
3.Dr. Seuss' The Lorax,Animation|Fantasy|Musical|IMAX
4.Aladdin and the King of Thieves,Animation|Children|Comedy|Fantasy|Musical|Romance
5.Cinderella,Animation|Children|Fantasy|Musical|Romance
6.Princess and the Frog, The,Animation|Children|Fantasy|Musical|Romance
7.Cloudy with a Chance of Meatballs,Animation|Children|Fantasy|IMAX
8.Christmas Carol, A,Animation|Children|Drama|Fantasy|IMAX
9.Frosty the Snowman,Animation|Children|Comedy|Fantasy|Musical
10.Enchanted,Adventure|Animation|Children|Comedy|Fantasy|Musical|Romance


In [ ]:
# Define a threshold for positive interactions
threshold = 3.5

# Create ground truth labels (1 for positive interaction, 0 for negative interaction)
df['ground_truth'] = (df['rating'] >= threshold).astype(int)

# Create a dictionary of ground truth interactions for each user
ground_truth_dict = df.groupby('userId')['ground_truth'].apply(list).to_dict()

In [ ]:
# Create an empty dictionary to store recommendations for each user
recommendations_dict = {}

for user_id, user_data in df.groupby('userId'):
    user_rated_movies = user_data['title'].tolist()
    recommendations = get_content_recommendations(user_rated_movies[0], cosine_sim=cosine_sim)
    recommendations_dict[user_id] = recommendations

In [ ]:

from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score, average_precision_score
from sklearn.metrics import ndcg_score

# Initialize lists to store evaluation metrics
map_list = []
ndcg_list = []


for user_id, recommendations in recommendations_dict.items():
    # Get ground truth interactions for the user
    ground_truth = ground_truth_dict.get(user_id, [])

    # Create a binary list indicating recommended items (1 if recommended, 0 otherwise)
    recommended_binary = [1 if movie in recommendations else 0 for movie in df['title']]

    # Pad recommended_binary with zeros if it's shorter than ground_truth
    if len(recommended_binary) < len(ground_truth):
        recommended_binary += [0] * (len(ground_truth) - len(recommended_binary))


     # Calculate Mean Average Precision (MAP)
    average_precision = average_precision_score(ground_truth, recommended_binary[:len(ground_truth)])

    # Calculate recall
    recall = recall_score(ground_truth, recommended_binary[:len(ground_truth)])

    # Calculate F1 score
    f1 = f1_score(ground_truth, recommended_binary[:len(ground_truth)])

    # Calculate accuracy
    accuracy = accuracy_score(ground_truth, recommended_binary[:len(ground_truth)])

    # Calculate Normalized Discounted Cumulative Gain (NDCG)
    ndcg = ndcg_score([ground_truth], [recommended_binary[:len(ground_truth)]])

    # Append metrics to lists
    map_list.append(average_precision)
    ndcg_list.append(ndcg)

# Calculate average metrics across all users

avg_map = sum(map_list) / len(map_list)
avg_ndcg = sum(ndcg_list) / len(ndcg_list)

print(f'Average Mean Average Precision (MAP): {avg_map}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Accuracy: {accuracy}')
print(f'Average Normalized Discounted Cumulative Gain (NDCG): {avg_ndcg}')


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Average Mean Average Precision (MAP): 0.6528757444560414
Recall: 0.0
F1 Score: 0.0
Accuracy: 0.28648233486943164
Average Normalized Discounted Cumulative Gain (NDCG): 0.8658187641198107


# Pickle files

In [ ]:
import pickle

In [ ]:
pickle.dump(cosine_sim ,open('cosine_sim.pkl','wb'))

